# Imports
We import some neccesary libraries and download some global scope data

In [45]:
import re
import os
import json
import pandas as pd
import ast
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Load the Datsets

In [46]:

def merge_csv_files_from_folder(folder_path):
    # Lista para almacenar los DataFrames
    df_list = []

    # Recorrer todos los archivos en la carpeta
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            # Obtener la ruta completa del archivo
            file_path = os.path.join(folder_path, filename)
            
            # Leer el archivo CSV y agregarlo a la lista de DataFrames
            df = pd.read_csv(file_path)
            df_list.append(df)

    # Concatenar todos los DataFrames en uno solo
    if df_list:
        merged_df = pd.concat(df_list, ignore_index=True)
        return merged_df
    else:
        print("No se encontraron archivos CSV en la carpeta.")
        return pd.DataFrame()  

data = merge_csv_files_from_folder("data_processed/csv/")
data.drop(columns="car_id", inplace=True)
colnames = data.columns.to_list()
for col in colnames: print(col)

C:\Users\marti\AppData\Local\Temp\ipykernel_18132\2912694450.py:12: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\marti\AppData\Local\Temp\ipykernel_18132\2912694450.py:12: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\marti\AppData\Local\Temp\ipykernel_18132\2912694450.py:12: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\marti\AppData\Local\Temp\ipykernel_18132\2912694450.py:12: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\marti\AppData\Local\Temp\ipykernel_18132\2912694450.py:12: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype op

price
km
year
color
cubicCapacity
brand
model
version
fuelType
bodyTypeId
transmissionTypeId
doors
seatingCapacity
warranty_months
province
environmentalLabel
co2Emissions
co2EmissionsGramsPerKm
trunkCapacityInLiters
maxSpeed
acceleration
hasDetails
jato_classification
traction
brakes
front_suspension
rear_suspension
power_cv
power_kw
rpm_max_power
max_torque_nm
rpm_max_torque
motor_description
transmission_description
speakers
trip_computer
remote_audio_control_on_steering_wheel
dvd_navigation
antenna
abs
electronic_traction_control
parking_sensors
airbag
stability_control
curve_braking_control
isofix_system
start_stop_automatic
cubicCapacity_seat_folded
Acabado Interior_Apoyabrazos
Acabado Interior_Asiento
Acabado Interior_Asientos
Acabado Interior_Dos
Acabado Interior_Pedales
Acabado Interior_Reloj
Acabado Interior_Reposacabezas
Acabado Interior_Tres
Acabado Interior_Uno
Confort_Aire
Confort_Apertura
Confort_Aviso
Confort_Banda
Confort_Calefacción
Confort_Cierre
Confort_Compartiment

In [47]:
for colum in [40,41,44,45,46,47]: data.iloc[:, colum] = data.iloc[:, colum].map({"True": True, "False": False})

C:\Users\marti\AppData\Local\Temp\ipykernel_18132\2833469390.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
165432    NaN
165433    NaN
165434    NaN
165435    NaN
165436    NaN
Name: cubicCapacity_seat_folded, Length: 165437, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  for colum in [40,41,44,45,46,47]: data.iloc[:, colum] = data.iloc[:, colum].map({"True": True, "False": False})


In [48]:
def find_dirºty_columns(prefix): return [col for col in data.columns if prefix in col]

# Cleaning Dirty Columns

There are four columns that require cleaning: `^Confort_`, `^Acabado Interior_` (already cleaned), `^Acabado Exterior_`, and `^electricFeatures`. You need to select one of these columns and process the data until it becomes readable. All columns should carry meaningful information. If a column contains a wide range of phrases, you can merge them into a single column labeled as "description," which will allow the application of NLP techniques later.

### Guidelines for Cleaning:

- **Singular and Plural Forms**:  
  If two columns represent the same feature, but one is the plural form of the other (e.g., `Confort_Luz` and `Confort_Luces`), discard the plural column.

- **Duplicate Column Names**:  
  If two columns share the same name, you can discard one of them.

- **Numbered Columns**:  
  If multiple columns are part of a series ending with a number (e.g., `Confort_Dos`, `Confort_Uno`), they likely describe different aspects of a single feature. In this case, merge them into a single column.

- **Relevance of Columns**:  
  Most of the dirty columns have names that describe the feature of their information, but some columns may be irrelevant.

- **Example for Reference**:  
  The column `^Acabado Interior_` has already been cleaned and can serve as an example of how to approach the cleaning process.

- **Nan and "no tiene"**:  
  Some data has an Nan and "no tiene", the first one means that we don't know the data but probably you can deduce that feature with other columns, specially in electrical features, and the second the car does nor have that feature.

**Data Types**:
  BE CAREFULL most of the cells that contains " ['Asiento de... " is a string not a list
  
At the beginning of each section, you will find a printout of the columns that need to be cleaned.

---

Let me know if you need further revisions or clarification!


## ~~Profe~~ **Lingfeng** te hacia `ilu` escribir esto no ? `XD` 

o espera

Prof Lingfeng you enjoyed writing this, right? XD

## Functions

In [49]:
def remove_duplicates(lista, elemento): return [x for x in lista if x != elemento]

def check_unique_values(column):
    ret = sorted(set(data[column].to_list()))
    print(f"Column: {column}")
    for x in ret: print(f'{x}: {data[column].to_list().count(x)}')
    print(f'NA: {data[column].to_list().count(None)}')

## Clean Confort


In [94]:
confort_columns = find_dirty_columns("Confort")
for x in confort_columns: print(x)


0                                                  No tiene
1               Sensores de aparcamiento traseros con radar
2               Sensores de aparcamiento traseros con radar
3         Sensores de aparcamiento delanteros con sensor...
4              Sensores de aparcamiento traseros con sensor
                                ...                        
165432          Sensores de aparcamiento traseros con radar
165433                                             No tiene
165434    Sensores de aparcamiento traseros con radar y ...
165435    Sensores de aparcamiento delanteros con sensor...
165436                                             No tiene
Name: parking_sensors, Length: 165437, dtype: object


TypeError: '<' not supported between instances of 'float' and 'str'

In [95]:
def classify_ac_zones(val):
    cleaned_val = re.sub(r"[\[\]']", '', val).strip().lower()

    match cleaned_val:
        case val if 'bizona' in val:
            return 'BIZONA'
        case val if 'trizona' in val or '3 zonas' in val:
            return 'TRIZONA'
        case val if '4 zonas' in val:
            return 'TETRAZONA'
        case _:
            return 'MONOZONA' 
        
def classify_ac_type(val): 
    cleaned_val = re.sub(r"[\[\]']", '', val).strip().lower()
    match cleaned_val:
        case val if 'semi-auto' in val or 'semi-automático' in val or 'semi-automatico' in val:
            return 'SEMIAUTO'
        case val if 'automático' in val or 'auto' in val or 'automatico' in val:
            return 'AUTO'
        case val if 'manual' in val:
            return 'MANUAL'
        case val if 'no tiene' in val:
            return 'NO_TIENE'
        case _:
            return 'STANDARD'  
        
def extraer_numero_tcinturones(text):
    if 'un' in text: return 1
    elif 'dos' in text:return 2
    elif 'tres' in text:return 3
    elif 'cuatro' in text:return 4
    elif 'cinco' in text:return 5
    elif 'seis' in text:return 6
    elif 'siete' in text:return 7
    elif 'Testigo' in text: return 2
    else: return 0
        

In [96]:
#Confort Aire
data["AC_Zones"] = data["Confort_Aire"].map(classify_ac_zones)
data["AC_Type"] = data["Confort_Aire"].map(classify_ac_type) 

#Confort_Apertura
#El maletero se abre a distancia? (Mecanicamente o electricamente)
data["trunk_auto_open"] = data["Confort_Apertura"].map(lambda x: False if 'no tiene' in x else True)

#Se podría quitar la verdad
data["BandaTintada"] = data["Confort_Banda"].map(lambda x: False if 'no tiene' in x else True)

data["Calefacción_Trasera"] = data["Confort_Calefacción"].map(lambda x: False if 'no tiene' in x else True)


data['Doble_Cierre'] = data['Confort_Cierre'].map(lambda x: True if 'doble cierre' in x else False)
data['Movil_NFC'] = data['Confort_Cierre'].map(lambda x: True if 'teléfono móvil' in x or 'NFC' in x else False)
data['Metodo_Apertura'] = data['Confort_Cierre'].map(
   lambda x: 'Mando a distancia' if 'mando a distancia' in x else
              'Tarjeta/llave inteligente' if 'tarjeta/llave inteligente' in x else
              'Llave' if 'llave' in x else
              'Teléfono móvil' if 'teléfono móvil' in x else
              'Desconocido'
)


data["Control_Crucero"] = data["Confort_Control"].map(lambda x: True if 'crucero' in x else False)
data["Control_Crucero_Adaptativo"] = data["Confort_Control"].map(lambda x: True if 'adaptativo' in x else False)  
data["SensoresDistancia"] = data["Confort_Control"].map(lambda x: True if 'distancia' in x else False)
data["StopGo"] = data["Confort_Control"].map(lambda x: True if 'stop' in x else False)


data["Dirección_Asistida"] = data["Confort_Dirección"].map(lambda x: True if 'dirección asistida' in x.lower() else False)
data["Dirección_Electrica"] = data["Confort_Dirección"].map(lambda x: True if 'eléctrica' in x.lower() else False)
data["Endurecimiento_Progresivo"] = data["Confort_Dirección"].map(lambda x: True if 'endurecimiento progresivo' in x.lower() else False)
data["Desmultiplicacion_Variable"] = data["Confort_Dirección"].map(lambda x: True if 'desmultiplicación variable' in x.lower() else False)
data["Dirección_Electro_Hidraulica"] = data["Confort_Dirección"].map(lambda x: True if 'electro-hidráulica' in x.lower() else False)


data["Elevalunas_Electricos"] = data["Confort_Elevalunas"].map(lambda x: True if 'elevalunas eléctricos' in x.lower() else False)
data["Un_Solo_Toque"] = data["Confort_Elevalunas"].map(lambda x: True if 'uno de ellos de un solo toque' in x.lower() or 'dos de ellos de un solo toque' in x.lower() else False)
data["Elevalunas_Traseros"] = data["Confort_Elevalunas"].map(lambda x: True if 'traseros' in x.lower() else False)


data["Tipo_Faro"] = data["Confort_Faros"].map(lambda x: 
    'LED' if 'led' in x.lower() else
    'Xenón' if 'xenón' in x.lower() else
    'Halógeno' if 'halógeno' in x.lower() else
    'Otro')
data["Luz_Larga"] = data["Confort_Faros"].map(lambda x: True if 'luz larga' in x.lower() else False)
data["Faros_Dobles"] = data["Confort_Faros"].map(lambda x: True if 'dobles' in x.lower() else False)


data["Indicador_Baja_Presion_Neumaticos"] = data["Confort_Indicador"].map(lambda x: 
    True if 'baja presion' in x.lower() else False)
data["Visualizacion_Presion"] = data["Confort_Indicador"].map(lambda x: 
    True if 'visualización de presión' in x.lower() else False)
data["Indicador_Consumo"] = data["Confort_Indicador"].map(lambda x: 
    True if 'indicador de consumo' in x.lower() else False)


data["Inmovilizador"] = data["Confort_Inmovilizador"].map(lambda x: 
    True if 'Inmovilizador' in x else False)


data["Intermitentes_Laterales"] = data["Confort_Intermitentes"].map(lambda x:
    True if 'laterales' in x else False)


data["Lavafaros"] = data["Confort_Lavafaros"].map(lambda x:
    True if 'Lavafaros' in x else False)


data["Sensor_Lluvia"] = data["Confort_Limpiaparabrisas"].map(lambda x: 
    True if 'sensor de lluvia' in x.lower() else False)
data["Intermitencia_Automatica"] = data["Confort_Limpiaparabrisas"].map(lambda x: 
    True if 'intermitencia automática' in x.lower() else False)


data["Luces_Lectura"] = data["Confort_Luces"].map(lambda x: 
    True if 'luces de lectura' in x.lower() else False)
data["Luces_Antiniebla"] = data["Confort_Luces"].map(lambda x: 
    True if 'luces antiniebla' in x.lower() else False)
data["Luces_Cortesía"] = data["Confort_Luces"].map(lambda x: 
    True if 'luces de cortesía' in x.lower() else False)
data["Luces_Laterales"] = data["Confort_Luces"].map(lambda x: 
    True if 'luces laterales maniobras' in x.lower() else False)


data["Memoria_Asientos"] = data["Confort_Memoria"].map(lambda x: 
    True if 'posición' in x else False)


data["Techo_Electrico"] = data["Confort_Techo"].map(lambda x: 
    True if 'eléctrico' in x.lower() else False)
data["Techo_Inclinable"] = data["Confort_Techo"].map(lambda x: 
    True if 'inclinable' in x.lower() else False)
data["Techo_Deslizante"] = data["Confort_Techo"].map(lambda x: 
    True if 'deslizante' in x.lower() else False)
data["Control_Remoto"] = data["Confort_Techo"].map(lambda x: 
    True if 'control remoto' in x.lower() else False)

data["Testigo_Cinturones"] = data["Confort_Testigo"].map(lambda x: 
    True if 'testigo de cinturones' in x.lower() else False)

data["Numero_Testigos"] = data["Confort_Testigo"].map(extraer_numero_tcinturones)


# Feed para el modelo NLP
data["Confort_Volante_Descripción"] = data["Confort_Volante"]


C:\Users\Barak\AppData\Local\Temp\ipykernel_17376\1301228139.py:97: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["Techo_Deslizante"] = data["Confort_Techo"].map(lambda x:
C:\Users\Barak\AppData\Local\Temp\ipykernel_17376\1301228139.py:99: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["Control_Remoto"] = data["Confort_Techo"].map(lambda x:
C:\Users\Barak\AppData\Local\Temp\ipykernel_17376\1301228139.py:102: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many ti

In [97]:
#Eliminar todas las columas con descripciones iniciales
for col in confort_columns: 
    data.drop(columns=col, inplace=True)

## Clean Acabado Interior 

In [98]:
acabado_interior_columns = find_dirty_columns("Acabado Interior")

In [99]:
acabado_interior_columns.remove("Acabado Interior_Asientos")
data.drop(columns="Acabado Interior_Asientos", inplace=True)

#for x in acabado_interior_columns: print(x)
#data[acabado_interior_columns]

In [100]:
data["sports_pedals"] = data["Acabado Interior_Pedales"].map({
    "['Pedales deportivos']": True, 
    'no tiene': False
})
data["interior_clock"] = data["Acabado Interior_Reloj"].map({
    "['Reloj analógico']": 'analogico', 
    "['Reloj digital']": 'digital',
    "['Reloj']": 'regular',
    "no tiene": 'no tiene'
})

#check_unique_values("sports_pedals")
#check_unique_values("interior_clock")


C:\Users\Barak\AppData\Local\Temp\ipykernel_17376\3934963263.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["sports_pedals"] = data["Acabado Interior_Pedales"].map({
C:\Users\Barak\AppData\Local\Temp\ipykernel_17376\3934963263.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["interior_clock"] = data["Acabado Interior_Reloj"].map({


In [101]:

def process_arm_suppport_front(phrases: list) -> str:
    stop_words = set(stopwords.words('spanish'))
    stop_words.add(",")
    text = " # ".join(phrases)
    if not ('delantero' in text): return ("no tiene")
    text = [word for word in text.split() if word not in stop_words]
    if '#' in text: 
        ret = " ".join( text[ (text.index('delantero')+1):text.index('#')] )
        return "Tiene" if len(ret) == 0 else ret
    else: 
        ret = " ".join( text[(text.index('delantero')+1):len(text)] ) 
        return "Tiene" if len(ret) == 0 else ret

def process_arm_suppport_back(phrases: list) -> str:
    stop_words = set(stopwords.words('spanish'))
    stop_words.add(",")
    text = " # ".join(phrases)
    if not ('trasero' in text): return ("no tiene")

    text = [word for word in text.split() if word not in stop_words]

    ret = " ".join( text[(text.index('trasero')+1):len(text)] ) 
    return "Tiene" if len(ret) == 0 else ret 

data["arm_suppport_front"] = data["Acabado Interior_Apoyabrazos"].map(lambda x: x if x == 'no tiene' else process_arm_suppport_front(eval(x)) )
data["arm_suppport_back"] = data["Acabado Interior_Apoyabrazos"].map(lambda x: x if x == 'no tiene' else process_arm_suppport_back(eval(x)) )

check_unique_values("arm_suppport_front")
check_unique_values("arm_suppport_back")

C:\Users\Barak\AppData\Local\Temp\ipykernel_17376\2034675041.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["arm_suppport_front"] = data["Acabado Interior_Apoyabrazos"].map(lambda x: x if x == 'no tiene' else process_arm_suppport_front(eval(x)) )


Column: arm_suppport_front
Tiene: 117241
abatible: 98
ajustable: 1
caja integrada: 4238
no tiene: 43859
NA: 0
Column: arm_suppport_back
Tiene: 64993
abatible: 5
acceso maletero: 406
ajustable: 2
caja integrada: 1415
caja integrada acceso maletero: 559
no tiene: 98057
NA: 0


C:\Users\Barak\AppData\Local\Temp\ipykernel_17376\2034675041.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["arm_suppport_back"] = data["Acabado Interior_Apoyabrazos"].map(lambda x: x if x == 'no tiene' else process_arm_suppport_back(eval(x)) )


In [102]:
def process_seat_description(phrases: list) -> str: return ". ".join(phrases)

data["seat_description"] = data["Acabado Interior_Asiento"].map( lambda x: x if x == 'no tiene' else process_seat_description(eval(x)) )

C:\Users\Barak\AppData\Local\Temp\ipykernel_17376\3282028378.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["seat_description"] = data["Acabado Interior_Asiento"].map( lambda x: x if x == 'no tiene' else process_seat_description(eval(x)) )


In [103]:
data["Acabado Interior_Uno"] = data["Acabado Interior_Uno"].map( lambda x: "['no tiene']" if x == 'no tiene' else x )
data["Acabado Interior_Dos"] = data["Acabado Interior_Dos"].map( lambda x: "['no tiene']" if x == 'no tiene' else x )
data["Acabado Interior_Tres"] = data["Acabado Interior_Tres"].map( lambda x: "['no tiene']" if x == 'no tiene' else x )
data["Acabado Interior_Reposacabezas"] = data["Acabado Interior_Reposacabezas"].map( lambda x: "['no tiene']" if x == 'no tiene' else x )

data["Acabado Interior_Uno"] = data["Acabado Interior_Uno"].map(ast.literal_eval)
data["Acabado Interior_Dos"] = data["Acabado Interior_Dos"].map(ast.literal_eval)
data["Acabado Interior_Tres"] = data["Acabado Interior_Tres"].map(ast.literal_eval)
data["Acabado Interior_Reposacabezas"] = data["Acabado Interior_Reposacabezas"].map(ast.literal_eval)

In [104]:
data['head_supporter'] = data.apply(
    lambda row: ". ".join(
        remove_duplicates(row["Acabado Interior_Uno"] + row["Acabado Interior_Dos"] + row["Acabado Interior_Tres"] + row["Acabado Interior_Reposacabezas"], 'no tiene')
    ), 
    axis=1)
#check_unique_values("head_supporter")

for col in acabado_interior_columns: data.drop(columns= col, inplace=True)

C:\Users\Barak\AppData\Local\Temp\ipykernel_17376\550879491.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['head_supporter'] = data.apply(


## Clean Acabado Exterior

In [50]:
stopw = frozenset(stopwords.words('spanish')) | {","}

def cleanTry(phrases: list) -> str:
    if not phrases:
        return "no tiene"
    
    # Join phrases into a single string and clean punctuation
    text = " ".join(phrases)
    
    # Remove stopwords efficiently
    text = [word for word in text.split() if word.lower() not in stopw]

    # Return cleaned text or "no tiene" if empty
    return "no tiene" if len(text) == 0 else " ".join(text)


In [51]:
data.drop(columns=["Acabado Exterior_Maletero","Acabado Exterior_Pasos","Acabado Exterior_Maletero","Acabado Exterior_Defensa","Acabado Exterior_Guardabarros"], inplace=True)

In [58]:
acabado_exterior_columns = find_dirty_columns("Acabado Exterior")
patata = data[acabado_exterior_columns].apply(lambda x: cleanTry(eval(x)) if x != 'no tiene' else x)
for x in acabado_exterior_columns: 
    print(patata[x].value_counts())
    print(x)
    print("Descripcion --------------------------")
    print(data[x].describe())
    print("Counts--------------------------------")
    print(data[x].value_counts())
    print("CleanTry------------------------------")

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

## Electronic Features

In [107]:
print(data["fuelType"])
#check_unique_values("fuelType") Pone que hay un float en esa columna, solucionalo


0                   Gasolina
1                   Gasolina
2                   Gasolina
3                  Eléctrico
4                    Híbrido
                 ...        
165432                Diésel
165433              Gasolina
165434               Híbrido
165435    Híbrido enchufable
165436              Gasolina
Name: fuelType, Length: 165437, dtype: object


# Save file

In [108]:
max_file_size = 100 * 1024 * 1024
output_folder = "NLP/test_data"
def split_dataframe_to_csv(df, max_file_size, base_filename):
    file_index = 1
    chunk_size = len(df) // 7 
    while chunk_size > 0:
        start_row = 0
        while start_row < len(df):
            end_row = start_row + chunk_size
            chunk = df.iloc[start_row:end_row]
            filename = f"{output_folder}/{base_filename}_part{file_index}.csv"
            chunk.to_csv(filename, index=False)
            if os.path.getsize(filename) > max_file_size:
                chunk_size //= 2
                os.remove(filename)
            else:
                start_row = end_row
                file_index += 1

        if start_row >= len(df):
            break

split_dataframe_to_csv(data, max_file_size, base_filename="test")

NameError: name 'data' is not defined